# M3 - Imagen

Este módulo es el núcleo del sistema de visión artificial. Operativamente está compuesto por cuatro bloques funcionales que siguen el flujo recogido en la siguiente imagen:

```{figure} .././assets/Modulo-3.png
:name: Figura_WP1_imagen.5
:alt: UML del modúlo de visión artificial
:width: 25%
:align: center

Bloques funcionales del módulo de imagen (visión artificial)
```
El motor de este módulo se sustenta sobre la librería GrabCut


In [17]:
import cv2, json, math, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

IMG_PATH = "./P22C-ML_0_calibrated.png"
OUT_OVERLAY = "./sole_overlay_mm_v2.png"
OUT_MASK = "./sole_mask_v2.png"
OUT_CSV = "./sole_measurements_v2.csv"
SCALE_JSON = "./calibracion_px_mm.json"

In [11]:
def get_scale_mm_per_px():
    try:
        from get_mm_per_px import get_mm_per_px
        val = float(get_mm_per_px()); 
        if val > 0: return val
    except Exception:
        pass
    p = Path(SCALE_JSON)
    if p.exists():
        try:
            data = json.loads(p.read_text())
            val = float(data.get("mm_per_px", 0))
            if val > 0: return val
        except Exception:
            pass
    return None


In [12]:
def segment_with_grabcut(bgr):
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    v = clahe.apply(v)
    hsv_eq = cv2.merge([h,s,v])
    bgr_eq = cv2.cvtColor(hsv_eq, cv2.COLOR_HSV2BGR)

    hsv2 = cv2.cvtColor(bgr_eq, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([95, 120, 40], dtype=np.uint8)
    upper_blue = np.array([130, 255, 255], dtype=np.uint8)
    blue_mask = cv2.inRange(hsv2, lower_blue, upper_blue)

    edge_margin = 20
    bg_border = np.zeros(blue_mask.shape, np.uint8)
    bg_border[:edge_margin, :] = 255
    bg_border[-edge_margin:, :] = 255
    bg_border[:, :edge_margin] = 255
    bg_border[:, -edge_margin:] = 255

    gc_mask = np.full(blue_mask.shape, cv2.GC_PR_BGD, dtype=np.uint8)
    gc_mask[blue_mask > 0] = cv2.GC_BGD
    gc_mask[bg_border > 0] = cv2.GC_BGD

    hgt, wdt = gc_mask.shape
    cx0, cy0 = wdt//2, hgt//2
    rect_w, rect_h = int(wdt*0.40), int(hgt*0.55)
    x1, y1 = max(cx0-rect_w, 0), max(cy0-rect_h, 0)
    x2, y2 = min(cx0+rect_w, wdt-1), min(cy0+rect_h, hgt-1)
    gc_mask[y1:y2, x1:x2] = np.where(blue_mask[y1:y2, x1:x2] == 0,
                                     cv2.GC_PR_FGD, gc_mask[y1:y2, x1:x2])

    bg_model = np.zeros((1,65), np.float64)
    fg_model = np.zeros((1,65), np.float64)
    cv2.grabCut(bgr_eq, gc_mask, None, bg_model, fg_model, 7, cv2.GC_INIT_WITH_MASK)

    bin_mask = np.where((gc_mask == cv2.GC_FGD) | (gc_mask == cv2.GC_PR_FGD), 1, 0).astype('uint8')
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
    bin_clean = cv2.morphologyEx(bin_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    bin_clean = cv2.morphologyEx(bin_clean, cv2.MORPH_CLOSE, kernel, iterations=2)
    return bin_clean

In [13]:
def pick_sole_contour(bin_mask):
    cnts, _ = cv2.findContours((bin_mask*255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not cnts:
        raise RuntimeError("No se detectaron contornos; ajusta semillas/umbrales.")
    def contour_score(c):
        area = cv2.contourArea(c)
        if area < 500: return -1
        (w,h) = cv2.minAreaRect(c)[1]
        if w == 0 or h == 0: return -1
        aspect = max(w,h)/max(1.0, min(w,h))
        return area + 2000.0*max(0.0, aspect-1.5)
    return max(cnts, key=contour_score)

In [14]:
def pca_measures_from_contour(cnt):
    pts = cnt.reshape(-1,2).astype(np.float32)
    mean, eigvecs, eigvals = cv2.PCACompute2(pts, mean=None)
    mean = mean.flatten()
    R = np.asarray(eigvecs, dtype=np.float32)
    pts_c = pts - mean
    pts_r = pts_c @ R.T
    min_x, max_x = np.min(pts_r[:,0]), np.max(pts_r[:,0])
    min_y, max_y = np.min(pts_r[:,1]), np.max(pts_r[:,1])
    length_px = (max_x - min_x)
    width_px  = (max_y - min_y)
    # eje mayor
    i_max = np.argmax(pts_r[:,0]); i_min = np.argmin(pts_r[:,0])
    p1 = (np.array([pts_r[i_min,0], 0.0], dtype=np.float32) @ R) + mean
    p2 = (np.array([pts_r[i_max,0], 0.0], dtype=np.float32) @ R) + mean
    # caja orientada
    box_r = np.array([[min_x,min_y],[max_x,min_y],[max_x,max_y],[min_x,max_y]], dtype=np.float32)
    box_img = (box_r @ R) + mean
    box_img = np.int32(box_img)
    return length_px, width_px, p1, p2, box_img

In [15]:
# === Ejecutar ===
bgr = cv2.imread(IMG_PATH, cv2.IMREAD_COLOR)
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

mask = segment_with_grabcut(bgr)
cnt = pick_sole_contour(mask)
length_px, width_px, p1, p2, box_img = pca_measures_from_contour(cnt)

In [16]:
mask = segment_with_grabcut(bgr)
cnt = pick_sole_contour(mask)
length_px, width_px, p1, p2, box_img = pca_measures_from_contour(cnt)

# Área en píxeles^2 a partir del contorno
area_px2 = float(cv2.contourArea(cnt))

# Overlay
overlay = rgb.copy()
cv2.drawContours(overlay, [cnt], -1, (0,255,0), 3)           # perímetro
cv2.polylines(overlay, [box_img], True, (255,255,0), 2)      # caja PCA
cv2.line(overlay, tuple(np.int32(p1)), tuple(np.int32(p2)), (255,0,0), 2)

cv2.imwrite(OUT_MASK, (mask*255))
cv2.imwrite(OUT_OVERLAY, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))

# Escala
mm_per_px = get_scale_mm_per_px()
if mm_per_px is not None and mm_per_px > 0:
    length_mm = length_px * mm_per_px
    width_mm  = width_px * mm_per_px
    area_mm2  = area_px2 * (mm_per_px**2)
else:
    length_mm = float("nan"); width_mm = float("nan"); area_mm2 = float("nan")

# Exportar CSV con área incluida
df = pd.DataFrame([{
    "image_path": IMG_PATH,
    "length_px": float(length_px),
    "width_px": float(width_px),
    "area_px2": float(area_px2),
    "mm_per_px": (float(mm_per_px) if mm_per_px else float("nan")),
    "length_mm": float(length_mm),
    "width_mm": float(width_mm),
    "area_mm2": float(area_mm2)
}])
df.to_csv(OUT_CSV, index=False)

print("Mediciones actualizadas:")
print(df.to_string(index=False))
print(f"\nOverlay: {OUT_OVERLAY}")
print(f"Máscara: {OUT_MASK}")
print(f"CSV:     {OUT_CSV}")

Mediciones actualizadas:
                image_path  length_px   width_px  area_px2  mm_per_px  length_mm  width_mm  area_mm2
./P22C-ML_0_calibrated.png 625.009644 287.143677  135660.0       0.05  31.250483 14.357184    339.15

Overlay: ./sole_overlay_mm_v2.png
Máscara: ./sole_mask_v2.png
CSV:     ./sole_measurements_v2.csv
